In [1]:
#!/usr/bin/env python3

##################################################################
## For the given list of proteins print out only the interactions
## between these protein which have medium or higher confidence
## experimental score
##
## Requires requests module:
## type "python -m pip install requests" in command line (win)
## or terminal (mac/linux) to install the module
##################################################################

import requests ## python -m pip install requests
import pandas as pd
import sys
import os

#path = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))+'\\Prepared_tables\\'
path = os.path.dirname(os.path.abspath('__file__'))+'\\Prepared_tables\\'


string_api_url = "https://string-db.org/api"
output_format = "tsv-no-header"
method = "network"
header = ['stringId_A', 'stringId_B', 'preferredName_A', 'preferredName_B', 'ncbiTaxonId', 'score', 'nscore', 'fscore', 'pscore', 'ascore', 'escore', 'dscore', 'tscore']
StringID = pd.DataFrame(columns= header)
##
## Construct URL
##

request_url = "/".join([string_api_url, output_format, method])

##
## Set parameters
##

#df_prot = pd.read_csv((path+ 'Prep_ProteinGroups_allproteins_75.csv'))

#df_prot = pd.read_csv((path+ 'Prep_Protein Groups_mito.csv'))
df_prot = pd.read_csv((path+ 'Prep_Protein Groups_cut05_notimuted.csv'))
df_prot = df_prot.iloc[[i for i in range(len(df_prot)) if str(df_prot["Pathways"][i]) != "nan"]].reset_index(drop= True)
df_prot['StringID'] = [list(set(str(df_prot['StringID'][i]).split(';'))) for i in range(len(df_prot))]
flatList = [ item for elem in list(df_prot['StringID']) for item in elem]
my_genes = sorted(list(set(flatList)))[1:-1]


params = {

    "identifiers" : "%0d".join(my_genes), # your protein
    "species" : 10090, # species NCBI identifier 
    "caller_identity" : "www.awesome_app.org" # your app name

}

##
## Call STRING
##

response = requests.post(request_url, data=params)

for line in response.text.strip().split("\n"):

    l = line.strip().split("\t")
    p1, p2 = l[2], l[3]
    
    # filter the interaction according to experimental score
    experimental_score = float(l[10])
    score = float(l[5])
    if score >= 0:
        StringID.loc[len(StringID)]=l



StringID = StringID.drop_duplicates()        
StringID.to_csv(path+'EdgeTable.csv', index= False)